In [2]:
# Author: Michael Bramble | michael.s.bramble@jpl.nasa.gov
# Investigating EMIT L2B MIN products for acid mind drainage 
# 20240705 - new version to incorporate fit/uncertainty data
# 20240708 - completed minimum working example

import numpy as np
import xarray as xr

# # point to the EMITL2BMIN product
# granule_asset_id_min = 'EMIT_L2B_MIN_001_20231015T215344_2328814_020.nc'
# fp_min = f'/Users/bramble/Documents/emit/alum_gulch/{granule_asset_id_min}'

# # point to the corresponding reflectance product
# granule_asset_id_ref = 'EMIT_L2A_RFL_001_20231015T215344_2328814_020.nc'
# fp_ref = f'/Users/bramble/Documents/emit/alum_gulch/{granule_asset_id_ref}'

# point to the corresponding uncertainty product for L2B_MIN data set
granule_asset_id_mun = 'EMIT_L2B_MINUNCERT_001_20231015T215344_2328814_020.nc'
fp_mun = f'/Users/bramble/Documents/emit/alum_gulch/{granule_asset_id_mun}'

# import the emit_tools module
import sys
sys.path.append('/Users/bramble/My Drive/_JPL_AMD/EMIT-Data-Resources-main/python/modules/')
from emit_tools import emit_xarray

# # import min data using the emit tools
# ds_geo_min = emit_xarray(fp_min, ortho=True)
# # import ref data using the emit tools
# ds_geo_ref = emit_xarray(fp_ref, ortho=True)
# # remove the bad bands
# ds_geo_ref['reflectance'].data[:,:,ds_geo_ref['good_wavelengths'].data==0] = np.nan

# import uncertainty data using the emit tools
ds_geo_mun = emit_xarray(fp_mun, ortho=True)

# select the group 1 fit data
band_fit_group_1 = ds_geo_mun.group_1_fit

# select just the spatial subset around a target of interst
max_lat = 31.55
min_lat = 31.45
max_lon = -110.8
min_lon = -110.7
slice_data_fit = band_fit_group_1.sel({'latitude' : slice(max_lat, min_lat),
                      'longitude' : slice(max_lon, min_lon)})
band_lon =  slice_data_fit['longitude']
band_lat =  slice_data_fit['latitude']

# array with classified mineral groups 
newarray_group_1 = xr.DataArray(None, slice_data_fit.coords, slice_data_fit.dims)

# # #
# perform mineral classification using newarray_group_1
# # #

# as with band depth quality check, perform quality check using fit data
for i in np.arange(0,len(band_lat)-1):
    for j in np.arange(0,len(band_lon)-1):
        match = band_fit_group_1[i,j]
        if match.values < 0.40: 
            newarray_group_1[i,j] = 0

# # #
# export newarray_group_1
# # #
# perform same process on group 2 data
# # #